### Progetto 8b
Luca Maria Lauricella 519997

Valerio Marini 512489

Repository del progetto:
https://github.com/lauriluca99/TGW-3D.jl

Documentazione del progetto:
https://lauriluca99.github.io/TGW-3D.jl

In [1]:
using BenchmarkTools
using SparseArrays
using Distributed
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

    function frag_face_channel(
            in_chan, 
            out_chan, 
            V::Points, 
            EV::ChainOp, 
            FE::ChainOp, 
            sp_idx::Vector{Int64})
			
Funziona che parallelizza, con l'utilizzo dei canali, la frammentazione delle facce in `FE` rispetto le facce in `sp_idx`.


In [2]:
function frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)
    run_loop = true
    while run_loop 
        
        sigma = take!(in_chan)
        
        if sigma != -1
            put!(out_chan,Lar.Arrangement.frag_face(V, EV, FE, sp_idx, sigma))
        else
            run_loop = false
        end
    end
    return V,EV
end

frag_face_channel (generic function with 1 method)

In [3]:
V = Float64[
    0 0 0; 0 1 0;
    1 1 0; 1 0 0;
    0 0 1; 0 1 1;
    1 1 1; 1 0 1
]


8×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  1.0  0.0
 1.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  1.0
 1.0  1.0  1.0
 1.0  0.0  1.0

In [4]:
EV = sparse(Int8[
    -1  1  0  0  0  0  0  0;
    0 -1  1  0  0  0  0  0;
    0  0 -1  1  0  0  0  0;
    -1  0  0  1  0  0  0  0;
    -1  0  0  0  1  0  0  0;
    0 -1  0  0  0  1  0  0;
    0  0 -1  0  0  0  1  0;
    0  0  0 -1  0  0  0  1;
    0  0  0  0 -1  1  0  0;
    0  0  0  0  0 -1  1  0;
    0  0  0  0  0  0 -1  1;
    0  0  0  0 -1  0  0  1;
])


12×8 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
 -1   1   ⋅   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅  -1   1   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅   ⋅  -1   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   ⋅   1   ⋅   ⋅  ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1   ⋅  ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  ⋅
  ⋅   ⋅   ⋅  -1   ⋅   ⋅   ⋅  1
  ⋅   ⋅   ⋅   ⋅  -1   1   ⋅  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅  -1   1  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  1
  ⋅   ⋅   ⋅   ⋅  -1   ⋅   ⋅  1

In [5]:
FE = sparse(Int8[
    1  1  1 -1  0  0  0  0  0  0  0  0;
    0  0  0  0  0  0  0  0 -1 -1 -1  1;
    -1  0  0  0  1 -1  0  0  1  0  0  0;
    0 -1  0  0  0  1 -1  0  0  1  0  0;
    0  0 -1  0  0  0  1 -1  0  0  1  0;
    0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

In [10]:
fs_num = size(FE, 1)
    
FV = Lar.compute_FV( EV, FE )
model = (convert(Lar.Points,V'), FV)
sp_idx = Lar.spaceindex(model)

in_chan = Distributed.RemoteChannel(()->Channel{Int64}(fs_num*2))
out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(10))


@async begin
    for sigma2 in 1:fs_num
        put!(in_chan, sigma2)
    end
    for p in Distributed.workers()
        put!(in_chan, -1)
    end
    for p in Distributed.workers()
        @async Distributed.remote_do(
        frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
    end
end

@time frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)
#@btime frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)

  0.021166 seconds (72.19 k allocations: 3.680 MiB, 76.33% compilation time)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 5, 1, 2, 6, 2, 3, 7, 3  …  12, 6, 9, 10, 7, 10, 11, 8, 11, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, 1, -1, 1, 1, 1], 12, 8))